# Binding prediction using AptaNet
Step-by-step guide for using AptaNet for binary aptamer–protein binding prediction.

## Overview
This notebook showcases the AptaNet algorithm, a deep learning method that combines sequence-derived (k-mer + PSeAAC) features with RandomForest-based feature selection, and a multi-layer perceptron to predict whether an aptamer and a protein interact (binary classification: aptamer binds/does not bind with the target protein). An overview of the classes and helper functions used in this notebook:

- **pairs_to_features**: helper that converts `(aptamer_sequence, protein_sequence)` pairs into feature vectors using k-mer + PSeAAC.
- **SkorchAptaNet**: a PyTorch MLP wrapped in Skorch for binary classification.
- **load_1gnh_structure**: helper to load the 1GNH molecule structure from PDB file.

## Data preparation
To train the `AptaNetMLP` the notebook uses:
* For `X`:
    * 5 random aptamer sequences of length>30 (to satisfy the default lambda value of 30 set in the PSeAAC algorithm).
    * Amino acid sequences extracted from the 1GNH protein molecule.
    
    The aptamer sequences and the amino acid sequences form tuples `(aptamer_sequence, protein_sequence)` to form `X`.
* For `y`:
    * A random binary value (to indicate if the aptamer binds or not) as dummy data.

In [ ]:
# Data imports
import torch

from pyaptamer.datasets import load_1gnh_structure
from pyaptamer.utils import struct_to_aaseq

In [24]:
aptamer_sequence = [
    "GGGAGGACGAAGACGACUCGAGACAGGCUAGGGAGGGA",
    "AAGCGUCGGAUCUACACGUGCGAUAGCUCAGUACGCGGU",
    "CGGUAUCGAGUACAGGAGUCCGACGGAUAGUCCGGAGC",
    "UAGCUAGCGAACUAGGCGUAGCUUCGAGUAGCUACGGAA",
    "GCUAGGACGAUCGCACGUGACCGUCAGUAGCGUAGGAGA",
]

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

# Build all combinations (aptamer, protein)
X = [(a, p) for a in aptamer_sequence for p in protein_sequence]

# Dummy binary labels for the pairs
y = torch.randint(0, 2, (len(X),), dtype=torch.float32)

## Building the pipeline
### Dataset balancing using the Neighbourhood cleaning rule
In the `AptaNet` paper, the authors mention using the `NeighbourhoodCleaningRule` from `imblearn` in order to balance the dataset, as in their dataset they had more negative (0) values than positives (1).

 To build a scikit-learn pipeline, follow these steps:
1. Convert the input to the desired (aptamer_sequence, protein_sequence) format.
    * OPTIONAL: As mentioned in the paper, perform under-sampling using the  
    Neighborhood Cleaning Rule to balance the classes.
2. Get the PSeAAC feature vectors for your converted input (using `pairs_to_features`).
3. Select the number of features to use from the feature vector (using `RandomForestClassifier`).
4. Define the skorch neural network (using `AptaNetMLP`).
### Different workflows
In this first half of the notebook we will cover 3 different workflows one can follow, in ascending order of customizability:

1. A minimal workflow with no dataset balancing, while using the in-built pipeline.
2. Using your own custom pipeline.
3. Dataset balancing, while using your own pipeline.

### First workflow
A minimal workflow with no dataset balancing, while using the in-built pipeline.

In [25]:
# If you want to use the AptaNet pipeline, you can import it directly
from pyaptamer.aptanet import AptaNetPipeline

In [26]:
pipeline = AptaNetPipeline()

In [27]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

In [28]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.5


### Second Worflow

Your own custom-built pipeline.

In [29]:
# If you want to build your own aptamer pipeline, you should use the following imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from skorch import NeuralNetBinaryClassifier

from pyaptamer.aptanet._aptanet_nn import AptaNetMLP
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [30]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

selector = SelectFromModel(
    estimator=RandomForestClassifier(
        n_estimators=300,
        max_depth=9,
        random_state=None,
    ),
    threshold="mean",
)

# Define the classifier
net = NeuralNetBinaryClassifier(
    module=AptaNetMLP,
    module__input_dim=None,
    module__hidden_dim=128,
    module__n_hidden=7,
    module__dropout=0.3,
    module__output_dim=1,
    module__use_lazy=True,
    criterion=torch.nn.BCEWithLogitsLoss,
    max_epochs=200,
    lr=0.00014,
    optimizer=torch.optim.RMSprop,
    optimizer__alpha=0.9,
    optimizer__eps=1e-08,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

pipeline = Pipeline(
    [
        ("features", feature_transformer),
        ("selector", selector),
        ("clf", net),
    ]
)

In [31]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7124       0.5000        0.6932  0.0000
      2        0.7718       0.5000        0.6931  0.0183
      3        0.7418       0.5000        0.6932  0.0000
      4        0.7457       0.5000        0.6932  0.0015
      5        0.6692       0.5000        0.6932  0.0156
      6        0.7629       0.5000        0.6932  0.0055
      7        0.6646       0.5000        0.6932  0.0103
      8        0.6563       0.5000        0.6932  0.0160
      9        0.7416       0.5000        0.6932  0.0000
     10        0.7760       0.5000        0.6932  0.0233
     11        0.7097       0.5000        0.6932  0.0089
     12        0.7042       0.5000        0.6932  0.0000
     13        0.6652       0.5000        0.6932  0.0157
     14        0.7385       0.5000        0.6932  0.0157
     15        0.6904       0.5000        0.6932  0.0073
     16        0.6981       0.5

In [32]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.5


### Third workflow
Dataset balancing using under-sampling, while using your own pipeline.

In [33]:
# If you want to build your own aptamer pipeline, you should use the following imports
from sklearn.preprocessing import FunctionTransformer

from pyaptamer.aptanet import AptaNetClassifier, AptaNetPipeline
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [34]:
# OPTIONAL: If you want to use the Neighborhood Cleaning Rule for under-sampling
# NOTE: If you want to use under-sampling, you need to install imbalanced-learn
# and use the Pipeline from imblearn
# %pip install imblearn
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [35]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

# AptaNetClassifier encapsulates the "selector" and "net" mentioned in Workflow 2
clf = AptaNetClassifier()

model = Pipeline(
    [
        ("ncr", NeighbourhoodCleaningRule()),
        ("clf", clf),
    ]
)

pipeline = AptaNetPipeline(estimator=model)

In [36]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

In [37]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.5


# Implementing `AptaNet` for real-world use cases

## Data preparation
To train the `AptaNetMLP` the notebook uses the dataset used to train the `AptaTrans` algorithm, this dataset can be found in `pyaptamer/datasets/data/train_li2014`.

In [38]:
# Data imports
import numpy as np

from pyaptamer.datasets import load_csv_dataset

In [ ]:
X_raw, y_raw = load_csv_dataset("train_li2014")

# Build combinations in the form of (aptamer, protein)
X = list(zip(X_raw.iloc[:, 0], X_raw.iloc[:, 1], strict=False))

# Dummy binary labels for the pairs
y = np.where(y_raw == "positive", 1, 0)

## DIfferent Real-world examples
In the second half of this notebook we will explore 3 real-world use cases of the algorithm. These include:
1. Training on the entire AptaTrans dataset, and predict binding probability (`predict_proba`) between a protein and a DNA sequence.
2. same for using DNA sequences from a `fasta` file.
3. Using MCTS combined with a trained `AptaNet` to propose new aptamers for a new pdb file, i.e., a form of in-silico Selex.

In [40]:
# Fit the pipeline on the aptamer-protein pairs from the `AptaTrans` dataset
pipeline.fit(X, y)

C:\Users\satvm\pyaptamer\pyaptamer\pseaac\_features.py:199: UserWarning: Invalid amino acid(s) found in sequence. Replaced with 'N'.
  seq = clean_protein_seq(protein_sequence)


### First workflow

Predicting binding probability (`predict_proba`) between a protein (1GNH) and a DNA sequence.

In [41]:
aptamer_sequence = "GGGAGGACGAAGACGACUCGAGACAGGCUAGGGAGGGA"

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

X = [(aptamer_sequence, p) for p in protein_sequence]

pipeline.predict_proba(X)

array([[0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ],
       [0.64597833, 0.3540217 ]], dtype=float32)

### Second workflow

Predicting binding probability (`predict_proba`) between a protein (1GNH) and DNA sequences from a `fasta` file.

In [42]:
from pyaptamer.utils import fasta_to_aaseq

fasta = fasta_to_aaseq(
    "https://huggingface.co/datasets/gcos/HoloRBP4_round8_trimmed/resolve/main/HoloRBP4_round8_trimmed.fasta"
)

X = [(aptamer_sequence, seq) for seq in fasta]

pipeline.predict_proba(X[:10])

array([[0.6472134 , 0.3527866 ],
       [0.64710045, 0.35289952],
       [0.6470602 , 0.35293978],
       [0.6473025 , 0.35269746],
       [0.6470673 , 0.35293266],
       [0.64710045, 0.35289952],
       [0.647261  , 0.352739  ],
       [0.64732355, 0.35267645],
       [0.64700234, 0.35299766],
       [0.647082  , 0.35291803]], dtype=float32)

### Third workflow

Using MCTS combined with a trained `AptaNet` to propose new aptamers for a protein (1GNH), i.e., a form of in-silico Selex.

In [43]:
from pyaptamer.experiments import AptamerEvalAptaNet
from pyaptamer.mcts import MCTS

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

# There can only be one target sequence
eval = AptamerEvalAptaNet(pipeline=pipeline, target=protein_sequence[0])

mcts = MCTS(experiment=eval)

mcts.run(verbose=True)


 ----- Round: 1 -----
##################################################
Best subsequence: C_U__A
Depth: 3
##################################################

 ----- Round: 2 -----
##################################################
Best subsequence: C_U__A_C_U_C
Depth: 6
##################################################

 ----- Round: 3 -----
##################################################
Best subsequence: C_U__A_C_U_C_C_G_U
Depth: 9
##################################################

 ----- Round: 4 -----
##################################################
Best subsequence: C_U__A_C_U_C_C_G_U_CA_G_
Depth: 12
##################################################

 ----- Round: 5 -----
##################################################
Best subsequence: C_U__A_C_U_C_C_G_U_CA_G_C_U_U_
Depth: 15
##################################################

 ----- Round: 6 -----
##################################################
Best subsequence: C_U__A_C_U_C_C_G_U_CA_G_C_U_U__C_AG_
Depth: 18
####

{'candidate': 'G',
 'sequence': 'C_U__A_C_U_C_C_G_U_CA_G_C_U_U__C_AG__C_G',
 'score': tensor([0.3545])}

In [45]:
# Reconstructing the best sequence
eval.reconstruct("C_U__A_C_U_C_C_G_U_CA_G_C_U_U__C_AG__C_G")

'GUUCGAUCACUCCGUCCACG'